In [ ]:
!cp '/content/drive/MyDrive/Colab Notebooks/tote256_17k_256_10k.zip' '/content/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/tote256_17k_256_10k.zip'

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
from PIL import Image

from keras.models import Sequential, load_model,Model

from tqdm.notebook import tqdm

In [ ]:
pip install torch===1.4.0 torchvision===0.5.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
"""
ConvTranspose2d:Applies a 2D transposed convolution operator over an input image composed of several input planes;

"""
class Generator(nn.Module):
    def __init__(self, nz=128, channels=3):
        super(Generator, self).__init__()
        
        self.nz = nz
        self.channels = channels
        
        def convlayer(n_input, n_output, k_size=4, stride=2, padding=0):
            block = [
                nn.ConvTranspose2d(n_input, n_output, kernel_size=k_size, stride=stride, padding=padding, bias=False),
                nn.BatchNorm2d(n_output),
                nn.ReLU(inplace=True),
            ]
            return block

        self.model = nn.Sequential(
            *convlayer(self.nz, 1024, 4, 1, 0), # Fully connected layer via convolution.
            *convlayer(1024, 512, 4, 2, 1),
            *convlayer(512, 256, 4, 2, 1),
            *convlayer(256, 128, 4, 2, 1),
            *convlayer(128, 64, 4, 2, 1),
            *convlayer(64, 32, 4, 2, 1),
            nn.ConvTranspose2d(32, self.channels, 4, 2, 1),
            nn.Tanh()
        )

    def forward(self, z):
        z = z.view(-1, self.nz, 1, 1)
        img = self.model(z)
        return img

class Discriminator(nn.Module):
    def __init__(self, channels=3):
        super(Discriminator, self).__init__()
        
        self.channels = channels

        def convlayer(n_input, n_output, k_size=4, stride=2, padding=0, bn=False):
            block = [nn.Conv2d(n_input, n_output, kernel_size=k_size, stride=stride, padding=padding, bias=False)]
            if bn:
                block.append(nn.BatchNorm2d(n_output))
            block.append(nn.LeakyReLU(0.2, inplace=True))
            return block

        self.model = nn.Sequential(
            *convlayer(self.channels, 32, 4, 2, 1),
            *convlayer(32, 64, 4, 2, 1),
            *convlayer(64, 128, 4, 2, 1, bn=True),
            *convlayer(128, 256, 4, 2, 1, bn=True),
            *convlayer(256, 512, 4, 2, 1, bn=True),
            *convlayer(512, 1024, 4, 2, 1, bn=True),
            nn.Conv2d(1024, 1, 4, 1, 0, bias=False),  # FC with Conv.  
        )



                
    def forward(self, imgs):
        out = self.model(imgs)
        return out.view(-1, 1)

In [ ]:
batch_size = 32
lr = 0.001
beta1 = 0.5
epochs = 30

real_label = 0.5
fake_label = 0
nz = 128

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
ROOT = "/content/drive/MyDrive/Colab Notebooks/model_cGan/"

In [ ]:
class DogDataset(Dataset):
    def __init__(self, img_dir, transform1=None, transform2=None):
    
        self.img_dir = img_dir
        self.img_names = os.listdir(img_dir)
        self.transform1 = transform1
        self.transform2 = transform2
        
        self.imgs = []
        for img_name in self.img_names:
            img = Image.open(os.path.join(img_dir, img_name))
            
            if self.transform1 is not None:
                img = self.transform1(img)
                
            self.imgs.append(img)

    def __getitem__(self, index):
        img = self.imgs[index]
        
        if self.transform2 is not None:
            img = self.transform2(img)
        
        return img

    def __len__(self):
        return len(self.imgs)

In [ ]:
transform1 = transforms.Compose([transforms.Resize((256,256))])

# Data augmentation and converting to tensors
random_transforms = [transforms.RandomRotation(degrees=5)]
transform2 = transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),
                                 transforms.RandomApply(random_transforms, p=0.3), 
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
                                 
train_dataset = DogDataset(img_dir='/content/tote256_17k_256_10k',
                           transform1=transform1,
                           transform2=transform2)

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=4)

                                           
imgs = next(iter(train_loader))
imgs = imgs.numpy().transpose(0, 2, 3, 1)

In [ ]:
fig = plt.figure(figsize=(25, 16))
for ii, img in enumerate(imgs):
    ax = fig.add_subplot(4, 8, ii + 1, xticks=[], yticks=[])
    
    plt.imshow((img+1)/2)

In [ ]:
import tensorflow.keras
from keras.utils import plot_model
import matplotlib.pyplot as plt

In [ ]:
netG = Generator(nz).to(device)
netD = Discriminator().to(device)

criterion = nn.BCELoss()

optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))


fixed_noise = torch.randn(25, nz, 1, 1, device=device)

In [ ]:
def show_generated_img():
    noise = torch.randn(1, nz, 1, 1, device=device)
    gen_image = netG(noise).to("cpu").clone().detach().squeeze(0)
    gen_image = gen_image.numpy().transpose(1, 2, 0)
    plt.imshow((gen_image+1)/2)
    plt.show()

In [ ]:
def train(batch_size = 32, epochs = 30, real_label = 0.5, nz = 128):
  for epoch in range(epochs):
    
      for ii, real_images in tqdm(enumerate(train_loader), total=len(train_loader)):
          ############################
          # (1) Update D network
          ###########################
          netD.zero_grad()
          real_images = real_images.to(device)
          batch_size = real_images.size(0)
          labels = torch.full((batch_size, 1), real_label, device=device)
          outputR = netD(real_images)
          noise = torch.randn(batch_size, nz, 1, 1, device=device)
          fake = netG(noise)
          outputF = netD(fake.detach())
          errD = (torch.mean((outputR - torch.mean(outputF) - labels) ** 2 ) + 
                  torch.mean((outputF - torch.mean(outputR) + labels) ** 2 ))/2
          errD.backward(retain_graph=True)
          optimizerD.step()
          ############################
          # (2) Update G network
          ###########################
          netG.zero_grad()
          outputF = netD(fake)   
          errG = (torch.mean((outputR - torch.mean(outputF) + labels) ** 2) +
                  torch.mean((outputF - torch.mean(outputR) - labels) ** 2))/2
          errG.backward()
          optimizerG.step()
          
          if (ii+1) % (len(train_loader)//2) == 0:
              print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f'
                    % (epoch + 1, epochs, ii+1, len(train_loader),
                      errD.item(), errG.item()))

      show_generated_img()  

In [ ]:
train(epochs = 30, batch_size=32)

In [ ]:
torch.save(netG.state_dict(), ROOT + "generator26_3.h5")
torch.save(netD.state_dict(), ROOT + "discriminator26_3.h5")

In [ ]:
netG.load_state_dict(torch.load(ROOT + "generator26_3.h5"))
netD.load_state_dict(torch.load(ROOT + "discriminator26_3.h5"))

<All keys matched successfully>

In [ ]:
gen_z = torch.randn(32, nz, 1, 1, device=device)
gen_images = (netG(gen_z).to("cpu").clone().detach() + 1)/2
gen_images = gen_images.numpy().transpose(0, 2, 3, 1)

In [ ]:
fig = plt.figure(figsize=(25, 16))
for ii, img in enumerate(gen_images):
    ax = fig.add_subplot(4, 8, ii + 1, xticks=[], yticks=[])
    plt.imshow(img)

In [ ]:
if not os.path.exists('/content/drive/MyDrive/Colab Notebooks/out_put_image256'):
    os.mkdir('/content/drive/MyDrive/Colab Notebooks/out_put_image256')
im_batch_size = 50
n_images=500
for i_batch in range(0, n_images, im_batch_size):
    gen_z = torch.randn(im_batch_size, nz, 1, 1, device=device)
    gen_images = (netG(gen_z) + 1)/2
    images = gen_images.to("cpu").clone().detach()
    images = images.numpy().transpose(0, 2, 3, 1)
    for i_image in range(gen_images.size(0)):
        save_image(gen_images[i_image, :, :, :], os.path.join('/content/drive/MyDrive/Colab Notebooks/out_put_image256', f'image_{i_batch+i_image:05d}.png'))

# torch.save(gen_images.state_dict(), '/content/model.json')

import shutil
shutil.make_archive('images', 'zip', '/content/drive/MyDrive/Colab Notebooks/out_put_image256')

In [ ]:
gen_z = torch.randn(32, nz, 1, 1, device=device)
gen_images = (netG(gen_z).to("cpu").clone().detach() + 1)/2
gen_images = gen_images.numpy().transpose(0, 2, 3, 1)

fig = plt.figure(figsize=(25, 16))
for ii, img in enumerate(gen_images):
    ax = fig.add_subplot(4, 8, ii + 1, xticks=[], yticks=[])
    plt.imshow(img)